In [15]:
import cv2
from tensorflow import *
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import Accuracy, Precision, Recall, F1Score
from keras import callbacks, layers, metrics, models, optimizers, regularizers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras.utils as image

In [2]:
train_path = "archive/chest_xray/train"
val_path = "archive/chest_xray/val"
test_path = "archive/chest_xray/test"

batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

validation_datagen = ImageDataGenerator(
    rescale = 1./255
)

test_datagen = ImageDataGenerator(
    rescale = 1./255
)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (128, 128),
    batch_size = batch_size,
    class_mode = "binary",
    color_mode = "grayscale"
)

validation_generator = validation_datagen.flow_from_directory(
    val_path,
    target_size = (128, 128),
    batch_size = batch_size,
    class_mode = "binary",
    color_mode = "grayscale"
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size = (128, 128),
    batch_size = batch_size,
    class_mode = "binary",
    color_mode = "grayscale"
)

train_num = train_generator.samples
val_num = validation_generator.samples

for image, label in train_generator:
    print(image.shape)
    print(label.shape)
    break

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
(32, 128, 128, 1)
(32,)


In [21]:
model = Sequential()

model.add(Conv2D(128, (3, 3), activation = "relu", input_shape = (128, 128, 1)))
model.add(MaxPooling2D((2, 2)))

model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation = "relu"))
model.add(MaxPooling2D((2, 2)))

model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation = "relu"))
model.add(MaxPooling2D((2, 2)))

model.add(BatchNormalization())
model.add(Conv2D(16, (3, 3), activation = "relu"))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, activation = "relu"))
model.add(Dense(1, activation = "sigmoid"))

model.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics = [Accuracy(), Precision(), F1Score()]
)

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 126, 126, 128)     1280      
                                                                 
 max_pooling2d_32 (MaxPooli  (None, 63, 63, 128)       0         
 ng2D)                                                           
                                                                 
 dropout_16 (Dropout)        (None, 63, 63, 128)       0         
                                                                 
 batch_normalization_24 (Ba  (None, 63, 63, 128)       512       
 tchNormalization)                                               
                                                                 
 conv2d_33 (Conv2D)          (None, 61, 61, 64)        73792     
                                                                 
 max_pooling2d_33 (MaxPooli  (None, 30, 30, 64)       

In [ ]:
epochs = 100

filepath = "Weights/weights.h5"

callbacks_list = [
    EarlyStopping(
        monitor = "val_acc",
        patience = 25
    ),
    ModelCheckpoint(
        filepath,
        monitor = "val_acc",
        verbose = 1,
        save_best_only = True,
        mode = 'max'
    )
]

history = model.fit(
    train_generator,
    steps_per_epoch = train_num / batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = val_num / batch_size,
    callbacks = callbacks_list
)

Epoch 1/100
163/163 [==============================] - ETA: 0s - loss: 0.2940 - acc: 0.8677 - precision_2: 0.9086 - f1_score: 0.8525
Epoch 1: val_acc improved from -inf to 0.62500, saving model to Weights\weights.h5
163/163 [==============================] - 148s 907ms/step - loss: 0.2940 - acc: 0.8677 - precision_2: 0.9086 - f1_score: 0.8525 - val_loss: 8.3673 - val_acc: 0.6250 - val_precision_2: 0.6250 - val_f1_score: 0.7692
Epoch 2/100
163/163 [==============================] - ETA: 0s - loss: 0.2951 - acc: 0.8685 - precision_2: 0.9119 - f1_score: 0.8525
Epoch 2: val_acc did not improve from 0.62500
163/163 [==============================] - 148s 907ms/step - loss: 0.2951 - acc: 0.8685 - precision_2: 0.9119 - f1_score: 0.8525 - val_loss: 6.7325 - val_acc: 0.6250 - val_precision_2: 0.6250 - val_f1_score: 0.7692
Epoch 3/100
163/163 [==============================] - ETA: 0s - loss: 0.2693 - acc: 0.8844 - precision_2: 0.9206 - f1_score: 0.8525
Epoch 3: val_acc did not improve from 0.62

In [ ]:
acc = history.history["acc"]
val_acc = history.history["val_acc"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, label = "Training loss")
plt.plot(epochs, val_loss, label = "Validation loss")
plt.legend()
plt.figure()

plt.plot(epochs, acc, label = "Training accuracy")
plt.plot(epochs, val_acc, label = "Validation accuracy")
plt.legend()
plt.figure()